# Misión 4: Evaluando y mejorando el robot bróker

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Recuerda poner en esta sección la ruta en la que guardaste los archivos correspondientes a los modelos que generamos en la misión anterior. 

In [ ]:
%cd "drive/MyDrive/RL_/"
!pwd

/content/drive/MyDrive/RL_
/content/drive/MyDrive/RL_


## Recapitulación de la misión anterior

In [ ]:
!pip install plotly
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 8.9 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
#!pip install tensorflow-gpu

In [ ]:
# Paquetes básicos para el análisis de datos
import math
import random
import numpy as np
import pandas as pd
import pandas_datareader as data_reader
from collections import deque

# Paquetes especializados en graficación e interacción
%matplotlib notebook
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Paquete especializado en tensores y, sobre todo, en redes neuronales
import tensorflow as tf
from tensorflow import keras

In [ ]:
def dataset_loader(stock_name, clip_data, train_portion=0.5):
    full_dataset = data_reader.DataReader(
        stock_name, data_source="yahoo").iloc[-clip_data:]

    data_start_date = str(full_dataset.index[0])
    data_end_date = str(full_dataset.index[-1])

    data = full_dataset['Close']
    data_samples = len(data)
    data_mid_date = str(full_dataset.index[int(data_samples * train_portion)])

    train_data = data[:data_mid_date]
    test_data = data[data_mid_date:]

    fmt_str = "{} <---[{}]---> {} <---[{}]---> {}"
    print("Timeline:\n", fmt_str.format(
        data_start_date.split()[0], len(train_data),
        data_mid_date.split()[0], len(test_data),
        data_end_date.split()[0]))

    return train_data, test_data

In [ ]:
class Trading_Environment():
    def __init__(self, dataset, verbose=False):
        self.window_size = 11
        self.state_size = self.window_size
        self.action_space = {0: "Stay", 1: "Buy", 2: "Sell"}

        self.data = dataset
        self.data_length = len(dataset)

        self.inventory = None
        self.current_step = None

        self.verbose = verbose

        self.reset()

    def reset(self):
        self.inventory = []
        self.current_step = 0

        return self._get_state()

    def step(self, action):
        reward = 0  # Only increases when selling is positive
        profit = 0

        current_price = self.data[self.current_step]

        if action == 1: # Buying
            self.inventory.append(current_price)

            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Compró", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        elif action == 2 and len(self.inventory) > 0: # Selling
            buy_price = self.inventory.pop(0)

            difference = current_price - buy_price

            reward = difference  # max(difference, 0)
            profit = difference
            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Vendió", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        else: # Staying
            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Esperó", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        self.current_step += 1
        next_state = self._get_state()

        if self.current_step >= self.data_length - 1:
            done = True
        else:
            done = False

        return next_state, reward, done, profit

    def _get_state(self):
        # Calculate the starting id
        starting_id = self.current_step - self.window_size + 1

        # Windowing data
        if starting_id >= 0:
            windowed_data = self.data[starting_id : self.current_step + 1].tolist()
        else:
            windowed_data = -starting_id * [self.data[0]] + \
                            list(self.data[0 : self.current_step + 1])

        # Normalise price data with sigmoid function
        state = 1. / (1. + np.exp(-np.diff(windowed_data)))

        # Return observation / state
        return np.concatenate(([len(self.inventory) * self.current_value], state))

    def get_action_sample(self):
        return random.randrange(len(self.action_space))

    @property
    def current_value(self):
        return self.data[self.current_step]

    @staticmethod
    def stock_price_format(n):
        return "- $ {:10f}, ".format(abs(n)) if n < 0 else "+ $ {:10f},".format(abs(n))
    

In [ ]:
class DQL_Agent():
    def __init__(self, environment, agent_name="Q_model", pretrained=False):
        self.env = environment
        self.memory = deque(maxlen=2000)
        self.batch_size = 32
        self.agent_name = agent_name

        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.first_step = True

        self.model = self.load_model() if pretrained else self.create_model()


    def create_model(self):
        model = keras.models.Sequential(name=self.agent_name)

        model.add(keras.Input(shape=(self.env.state_size,)))
        #model.add(keras.layers.Dense(name="Layer1", units=128, activation='relu'))
        #model.add(keras.layers.Dense(name="Layer2", units=256, activation='relu'))
        #model.add(keras.layers.Dense(name="Layer3", units=256, activation='relu'))
        model.add(keras.layers.Dense(name="Layer4", units=32, activation='relu'))
        model.add(keras.layers.Dense(name="Layer5", units=len(self.env.action_space), activation='linear'))

        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def get_action(self, s, is_evaluating=False):
        if self.first_step:
            self.first_step = False
            return 1

        if not is_evaluating and random.random() <= self.epsilon:
            return self.env.get_action_sample()

        else:
            return int(tf.argmax(self.model.predict(
                s.reshape(1, self.env.state_size), verbose=0)[0]))

    def remember(self, sarsd):
        # sarsd: (state, action, reward, new_state, done)
        self.memory.append(sarsd)

    def experience_replay(self):
        batch_sample = random.sample(self.memory, self.batch_size)

        states = np.array([sarsd[0] for sarsd in batch_sample])
        actions = np.array([sarsd[1] for sarsd in batch_sample])
        rewards = np.array([sarsd[2] for sarsd in batch_sample])
        next_states = np.array([sarsd[3] for sarsd in batch_sample])
        done = np.array([sarsd[4] for sarsd in batch_sample])

        # Q predicho
        target = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        target[done] = rewards[done]

        # Q actual
        q_values = self.model.predict(states, verbose=0)
        q_values[range(self.batch_size), actions] = target

        loss = self.model.fit(states, q_values, epochs=1, verbose=0).history["loss"][0]

        """ Slow version:
        states_train = []
        q_train = []

        for sarsd in batch_sample:
            state, action, reward, new_state, done = sarsd

            target = reward
            if not done:
                future_rewards = self.model.predict(new_state.reshape(1, self.env.state_size), verbose=0)[0]
                target += self.gamma * np.amax(future_rewards)

            q_values = self.model.predict(state.reshape(1, self.env.state_size), verbose=0)

            q_values[0][action] = target

            states_train.append(state)
            q_train.append(q_values[0])

        loss = self.model.fit(np.stack(states_train), np.stack(q_train), epochs=1, verbose=0).history["loss"][0]
        """

        self.update_epsilon()

        return loss

    def update_epsilon(self):
        if self.epsilon >= self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self, episode):
        self.model.save("models/{}_{}".format(self.agent_name, episode))

    def load_model(self):
        return keras.models.load_model("models/" + self.agent_name)

In [ ]:
def train_model(agent, env, num_episodes=10):
    profit_evolution = []
    reward_evolution = []
    average_loss = []

    for episode in range(1, num_episodes + 1):

        total_profit = 0
        total_reward = 0
        loss_list = []

        state = env.reset()
        done = False
        while not done:
            action = agent.get_action(state)
            new_state, reward, done, profit = env.step(action)

            sarsd = (state, action, reward, new_state, done)
            agent.remember(sarsd)

            if len(agent.memory) > agent.batch_size:
                loss = agent.experience_replay()
                loss_list.append(loss)

            state = new_state

            total_profit += profit
            total_reward += reward

            #print("Step: {}, Loss: {}, Profit: {}".format(env.current_step, loss, profit))

        print(f"Episode {episode}, Profit: {total_profit}, Reward: {total_reward}")

        profit_evolution.append(total_profit)
        reward_evolution.append(total_reward)
        if len(loss_list) > 0:
            average_loss.append(np.mean(np.array(loss_list)))

        if episode % 10 == 0:
            agent.save_model(f"{episode}")

    agent.save_model(f"{num_episodes}")
    return profit_evolution, reward_evolution, average_loss

## Ejercicio 1


In [ ]:
def test_model(agent, env, epsilon=None):
    profit_evolution = []
    reward_evolution = []
    history = []

    total_profit = 0
    total_reward = 0
    
    if epsilon is None:
        evaluation_flag = True
    else:
        evaluation_flag = False
        agent.epsilon_decay = 0.0
        agent.epsilon_min = 0.6

    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state, is_evaluating=evaluation_flag)
        new_state, reward, done, profit = env.step(action)

        sarsd = (state, action, reward, new_state, done)
        agent.remember(sarsd)

        state = new_state

        history.append((env.current_value, env.action_space[action]))

        total_profit += profit
        profit_evolution.append(total_profit)
        reward_evolution.append(reward)

        #print(f"Step {env.current_step}, Profit: {profit}, Reward: {reward}")
        #print("Step: {}, Profit: {}".format(env.current_step, profit))

    return profit_evolution, reward_evolution, history


In [ ]:
train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

entorno = Trading_Environment(test_data)
dql_agent = DQL_Agent(entorno, agent_name="robot_d2ql_FINAL", pretrained=True)

Timeline:
 2021-11-26 <---[101]---> 2022-04-21 <---[100]---> 2022-09-13


Prueba con una función greedy

In [ ]:
pev, rev, history = test_model(agent=dql_agent, env=entorno, epsilon=None)

In [ ]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution", 
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward", 
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices], 
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices], 
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices], 
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()



## Ejercicio 2

Prueba con una función 0.1-greedy

In [ ]:
pev, rev, history = test_model(agent=dql_agent, env=entorno, epsilon=0.01)

In [ ]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution", 
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward", 
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices], 
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices], 
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices], 
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()



## Ejercicio 3

In [ ]:
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)

entorno = Trading_Environment(test_data, verbose=False)
agente = DQL_Agent(entorno, agent_name="robot_d5ql_full_50", pretrained=True)


Timeline:
 2018-09-24 <---[501]---> 2020-09-18 <---[500]---> 2022-09-13


In [ ]:
agente.model.summary()

Model: "robot_d5ql_full"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 128)               1536      
                                                                 
 Layer2 (Dense)              (None, 256)               33024     
                                                                 
 Layer3 (Dense)              (None, 256)               65792     
                                                                 
 Layer4 (Dense)              (None, 32)                8224      
                                                                 
 Layer5 (Dense)              (None, 3)                 99        
                                                                 
Total params: 108,675
Trainable params: 108,675
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!pip install visualkeras
import visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 989 kB 6.7 MB/s 


In [ ]:
visualkeras.layered_view(agente.model)

In [ ]:
pev, rev, history = test_model(agent=agente, env=entorno, epsilon=0.01)

In [ ]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution", 
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward", 
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices], 
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices], 
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices], 
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


In [ ]:
_, test_data = dataset_loader(stock_name="IBM", clip_data=2000, 
                              train_portion=0.1)

entorno = Trading_Environment(test_data, verbose=False)
agente = DQL_Agent(entorno, agent_name="robot_d5ql_full_50", pretrained=True)


Timeline:
 2017-09-14 <---[126]---> 2018-03-15 <---[1133]---> 2022-09-13


In [ ]:
pev, rev, history = test_model(agent=agente, env=entorno, epsilon=0.01)

In [ ]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution", 
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward", 
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices], 
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices], 
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices], 
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()
